# Milestone 2
Supervised learning.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import sklearn
import sklearn.preprocessing
import sklearn.metrics

In [2]:
df = pd.read_csv('dengue_features_train.csv')

## Filter data for group subset
Our group is working on dataset San Juan 1992-98

In [3]:
df['city'].value_counts()

sj    936
iq    520
Name: city, dtype: int64

In [4]:
#criteria = (df['city'].values == 'sj', df['year'].values >= 1992, df['year'].values <= 1998)
#df = df[np.sum(criteria, axis=0, dtype=np.int8) == 3]
df = df[df['city'] == 'sj']
df = df[df['year'] >= 1992]
df = df[df['year'] <= 1998]

In [5]:
df.drop(['city', 'year', 'weekofyear', 'week_start_date'], axis=1, inplace=True)
df.shape

(364, 20)

In [6]:
df.head()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
87,0.125100,0.157867,0.185283,0.181367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,0.100833,0.073575,0.138714,0.112786,200.85,297.912857,298.121429,294.457143,300.4,296.1,18.30,81.300000,200.85,15.878571,2.300000,25.657143,7.500000,30.6,20.6,58.2
89,0.222400,0.229900,0.198457,0.199414,3.12,297.338571,297.485714,294.467143,300.2,295.6,19.80,84.117143,3.12,15.832857,1.857143,25.471429,6.171429,31.1,21.7,32.3
90,0.122533,0.131050,0.188843,0.168871,0.00,297.091429,297.335714,293.021429,299.5,295.6,5.11,78.291429,0.00,14.482857,2.357143,24.857143,7.442857,30.6,20.0,1.3
91,NaN,0.048300,0.173671,0.191929,0.00,297.420000,297.550000,292.718571,299.3,295.5,6.50,75.220000,0.00,14.185714,2.442857,24.800000,6.585714,28.9,20.0,9.2


# Fill NaN

In [7]:
df.isna().sum()
# ffill: propagate last valid observation forward to next valid backfill
# @ Check other methods
df.fillna(method="bfill", inplace=True)
df.isna().sum()

ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
dtype: int64

# PCA
Normalize the data and apply find best principal components.

In [9]:
scaler = sklearn.preprocessing.StandardScaler()
#scaler = sklearn.preprocessing.MinMaxScaler()
X = scaler.fit_transform(df)

In [10]:
from sklearn.decomposition import PCA
pca = PCA()
X_pca = pca.fit_transform(X)
X_pca.shape

(364, 20)

### Selected features

In [1]:
features = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_precip_amt_kg_per_m2', 'reanalysis_specific_humidity_g_per_kg', 'group']